In [ ]:
### Load libraries
library(here)
library(rstan)
library(rgdal)

In [ ]:
#### Define paths:

# Inputs
path2env_sd<- here("data","Sd_Env_by_county.csv")
path2prd_counties<-here("data","production","by_county")
path2adjacency <- here("data","adjacency.csv")
stan_model<-here("stan","parabolic_iCAR.stan")

# Outputs
path2models<-here("outputs","models")

In [ ]:
### Load data
env<-read.csv(path2env_sd,row.names=1) # predictors
adjacency<-read.csv(path2adjacency) # adjacency

In [ ]:
### Format data

# Get the adjacent nodes from the matrix:
node1<-adjacency$node1
node2<-adjacency$node2
N_edges<-length(node1)

# Define stan data:
y=prd$logProportionAreaCultivated
x1=env$BIO10_sd
x2=env$BIO17_sd
x3=env$BIO4_sd
x4=env$BIO9_sd
x5=env$npp_sd

### Define model parameters:

warmup=2000
iter=10000
chains=5
cores=5
thin=1


crop="rice"
path2model <- paste(path2models,"/",crop,".rds",sep="")
path2prd<-paste(path2prd_counties,"/",crop,".csv",sep="")

prd<-read.csv(path2prd,row.names=1) # response
y=prd$logProportionAreaCultivated
#Format the data
stan_data <- list(N = N, x1 = x1, y = y,x2=x2,x3=x3,x4=x4,x5=x5,N_edges=N_edges,node1=node1,node2=node2)

In [ ]:
crops<-c("wheat","barley")

In [ ]:
for (crop in crops){
    # Set crop specific paths
    print(crop)
    path2model <- paste(path2models,"/",crop,".rds",sep="")
    path2prd<-paste(path2prd_counties,"/",crop,".csv",sep="")
    # Load production data
    prd<-read.csv(path2prd,row.names=1) # response
    N<-nrow(prd)
    y=prd$logProportionAreaCultivated
    #Format the data
    stan_data <- list(N = N, x1 = x1, y = y,x2=x2,x3=x3,x4=x4,x5=x5,N_edges=N_edges,node1=node1,node2=node2)
    ### Run model
    fit <- stan(file = stan_model, data = stan_data, warmup = warmup, iter = iter, chains = chains, cores = cores, thin = thin,control = list(max_treedepth = 20))

    ### Save model
    saveRDS(fit, path2model) }